In [3]:
from bs4 import BeautifulSoup
import urllib
import requests
import csv
import pandas as pd
import bs4
import time
from datetime import datetime
from pprint import pprint

# --------- Information ---------
CITY = "Philadelphia"
GPS_COORDS = "39.9524,-75.1636"
DARKSKY_API_KEY = "aff5db3d6a9b4506d19657f6186174a6"
START_DATE = "20190520"
END_DATE = "20190522"
# --------------------------------- 

def bitcoin_plus_weather_history():
    datelist = []
    friendlydatelist = []
    highlist = []
    lowlist = []
    openlist = []
    closelist = []
    volumelist = []
    marketlist = []

    #########
    # Collect bitcoin data
    #########
    r = requests.get('https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=' + START_DATE + '&end=' + END_DATE)
    # r = requests.get('https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20140522&end=20190522')
    soup = bs4.BeautifulSoup(r.text, "lxml")
    tr = soup.find_all('tr', {'class': 'text-right'})
    for item in tr:
        user_friendly_date = item.find('td', {'class': 'text-left'}).text
        month = user_friendly_date[:3]
        day = user_friendly_date[4:7]
        year = user_friendly_date[8:]
        date = datetime.strptime(user_friendly_date, '%b %d, %Y')
        unixtime = int(time.mktime(date.timetuple())) # Need Unix time for DarkSky to compute

        datelist.append(unixtime)
        friendlydatelist.append(user_friendly_date)
        highlist.append(item.find_all('td')[2].text)
        openlist.append(item.find_all('td')[1].text)
        lowlist.append(item.find_all('td')[3].text)
        closelist.append(item.find_all('td')[4].text)
        volumelist.append(item.find_all('td')[5].text)
        marketlist.append(item.find_all('td')[6].text)
    row0=['datelist', 'openlist','highlist','lowlist','closelist','volumelist','marketlist']
    rows=zip(datelist,friendlydatelist,openlist,highlist,lowlist,closelist,volumelist,marketlist)
    
    ######### 
    # Merge bitcoin and weather data 
    #########
    for item in rows:
        row_unixtime = item[0]
        resp = requests.get('https://api.darksky.net/forecast/' 
                            + DARKSKY_API_KEY + '/' 
                            + GPS_COORDS + ',' 
                            + str(row_unixtime))

        # Check resp status
        if resp.status_code == 200:
            data = resp.json()
        else:
            raise ValueError("Check your Internet connection or API daily limit has been reached - try again tomorrow")

        if data:
            current_weather = data["currently"]
            if 'temperature' and 'summary' in current_weather:
                item = item[1:]
                item = item + (data["currently"]["temperature"], data["currently"]["summary"])
                print(item)
        else:
            raise ValueError("Something went wrong!?")


    # # For testing:
    # resp = requests.get('https://api.darksky.net/forecast/' 
    #                     + DARKSKY_API_KEY + '/' 
    #                     + GPS_COORDS + ',' 
    #                     + str(row_unixtime))
    # data = resp.json()
    # # pprint(data["daily"]["data"][0]["summary"])
    # print(data["currently"]["temperature"])
    # print(data["currently"]["summary"])
    # pprint(data)


bitcoin_plus_weather_history()


('May 22, 2019', '7956.29', '7997.61', '7615.99', '7680.07', '24,719,473,175', '136,063,898,960', 55.07, 'Clear')
('May 21, 2019', '7977.97', '8062.17', '7843.34', '7963.33', '25,127,245,056', '141,065,573,610', 66.39, 'Clear')
('May 20, 2019', '8196.92', '8200.97', '7678.78', '7978.31', '23,843,404,340', '141,315,695,728', 70.82, 'Partly Cloudy')


## What we would like to our data to do/hope it is good for
We will gather historical data (over the past 10 years) on Philadelphia, PA and Tucson, AZ.  We will then glean information on past daily precipitation and high and low temperatures. We will then scrape historical cryptocurrency data and merge the weather data to the historical cryptocurrency data to show what the weather was like on each day listed in the crypto currency historical data.

## Potential Data Users (who might be interested in your data set)
Potential users of this data are people who may want to see if weather across the country can be considred an external force to the amount of cryptocurrency that is bought or sold on any given day. Also, this would be a good set of data for anyone wanting to do historical cryptocurrency analysis and a weather analysis.


## Data Limitations and possible improvements

1. Performance:
    * Downstream/upstream issues
    * Each developer must request an API key, which allows the developer 1000 calls per day for free.  Given that we have four team members, we can gather 4000 days of data per day.  Since we want to collect 10 years of data, we will need to make 365*3*10 = 10950 calls.  If we used our calls optimally, that will take us less than a week to gather.  We’ll need to write a script to automate data gathering (make the request to Dark Sky, process the response for the information we want, and store that information in a usable format), so that we don’t have to manually take those actions.   
2. Data limitations:
    * Security protocols - user Agent is required to identify the application
    * Inconsistency in data may include incomplete, out of date, general errors, overall data sets could potentially be imprecise.
3. Data improvements:
    * Our data would be improved by adding more vacation destinations.  We will not do this during this project, but if we were to extend/commercialize the project, increasing the list of cities would increase the value of our product
    * Data documentation
    * Import and manipulate data into database for easy consumption.
    * Provide financial metrics such as average, median, beta etc.

## How was data created? (e.g., people texting, The Earth's molten core spinning, etc.)
The data on US weather is gathered from a number of weather monitoring sources, most of which are governmental.  NOAA is an important data source for Dark Sky.  Additional information on data sources can be found here: https://darksky.net/dev/docs/sources

The cryptocurrency data is gathered from all the various exchanges. The querey is updated every minute to provide the most up to dat data. It then runs through a cleaning process and through algorithms to ensure the data is correct. Additional information can be found here: https://coinmarketcap.com/methodology/

## Current and planned future data access rights
Dark Sky defines their data as their features, information, pictures and other contained on their site. The Site and the Data are protected by certain proprietary rights and laws. Dark Sky grants the license and permit for developers or organizations to access their data. The message “Powered by Dark Sky” in a legible manner shall be put near the data or any information derived from any data from the Dark Sky if user uses their Data. However, Dark Sky doesn’t allow users to sell, lease, or sublicense their Service, use their Service without authorization, and
interfere or attempt to interfere with the proper functioning of the Service.

No future plan on data access rights was provided in the documentation. However, the Term mentions that the Service may be updated from time to time for any reason. Dark Sky will notify users of any changes to their Terms of Service 

Additional information on Dark Sky’s terms of use can be found here: https://darksky.net/dev/docs/terms.

CoinMareketCap considers all data, headers, slogans, tradmarks, and picutres to be owned and controlled by CoinMarketCap. The data and all other features of the website are protected by certain proprietary rights and laws. CoinMarketCap does allow for limited use if the project is for your own personal use and will not be distrubuted commercially. The message "Powered by CoinMarketCap" in a legible manner shall be put near the data or any information derived from any data from CoinMarketCap if user uses their Data. 

No future plan on data access rights was provided in the documentation.

Additional information on CoinMarketCap's terms of use can be found here: https://coinmarketcap.com/terms/
